In [1]:
print("Hello Zuko Here")

Hello Zuko Here


In [58]:

from keras.models import load_model
from keras import backend as K
from keras import losses
import rasterio
import os
import numpy as np
import random
import tifffile

In [3]:

def iou_score(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    union = K.sum(y_true,-1) + K.sum(y_pred,-1) - intersection
    iou = (intersection + smooth) / ( union + smooth)
    return iou


def dice_coeff(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score

def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss

def bce_dice_loss(y_true, y_pred):
    loss = losses.binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    return loss


def bce_dice_loss_new(y_true, y_pred):
    return 0.8*losses.binary_crossentropy(y_true, y_pred) + 0.2*(1 - dice_loss(y_true, y_pred))

In [4]:
path ='D:\\jupyterlab\\omdena-silicon-valley-earthquakes\\original\\road_segmentation\\model_latest.h5'
model = load_model(path, custom_objects={'bce_dice_loss': bce_dice_loss, 'iou_score' : iou_score})

In [5]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 325, 325, 8) 0                                            
__________________________________________________________________________________________________
conv2d_52 (Conv2D)              (None, 320, 320, 16) 4624        input_4[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_13 (MaxPooling2D) (None, 160, 160, 16) 0           conv2d_52[0][0]                  
__________________________________________________________________________________________________
dropout_25 (Dropout)            (None, 160, 160, 16) 0           max_pooling2d_13[0][0]           
____________________________________________________________________________________________

In [76]:
img_path = 'D:\\jupyterlab\\omdena-silicon-valley-earthquakes\\hravat\\Budalangi_202005_after_all_bands.tif'
image = rasterio.open(img_path).read()
random.seed(10)   
image_cut = image[:,rand_int:rand_int+325,rand_int:rand_int+325]

image_pred = np.empty([325,325])

for i in range(7):
    image_pred=np.append(image_pred,image_cut[i], axis=-1)
    
image_pred=np.reshape(image_pred,(325,325,8))
print(np.shape(image_pred))


(325, 325, 8)


In [77]:
x_val=image_pred_final
predictions = model.predict(x_val)

In [78]:
print(np.shape(x_val))

(1, 325, 325, 8)


In [79]:
print(np.shape(predictions))

(1, 325, 325, 1)


In [80]:
type(predictions)

numpy.ndarray

In [81]:
tifffile.imwrite('my_image.tif', image_cut)

In [60]:
print(np.shape(image))

(13, 2326, 2060)


In [85]:
predictions = np.squeeze(predictions,axis=(0,-1))
print(np.shape(predictions))

(325, 325)


In [95]:
from PIL import Image as im
data = Image.fromarray(predictions)
if data.mode != 'RGB':
    data = data.convert('RGB')

In [96]:
data.save("predictions.jpeg")

In [98]:
np.min(predictions)

3.6824645e-13

In [99]:
np.max(predictions)

0.31177244